# Gridworld with First Visit

In [1]:
import numpy as np

# display output
from random import uniform
import random
import time
from IPython.display import display, clear_output

In [2]:
actions = [[-1, 0], [0, 1], [1, 0], [0, -1]] #up, right, down, left = (clockwise from up) 
action_count = len(actions) # total number of actions
gridSize = 5 # create a square grid of gridSize by gridSize
state_count = gridSize*gridSize # total number of states

In [3]:
class Gridworld():
    def __init__(self, gridSize):
        self.valueMap = np.zeros((gridSize, gridSize))
        self.states = [[i, j] for i in range(gridSize) for j in range(gridSize)]
        self.size = gridSize
        self.new_pos = [0, 0] # initialize new position for p_transition
        self.transition_prob = 1 # deterministic
    
    def initial_state(self):        # return initial state
        return grid.states[gridSize*gridSize-1]
   
    def transition_reward(self, current_pos, action): # return the transition probability

        # get next position: state: [0, 0], action: [0, 1], new_state = [0, 1]
        self.new_pos = np.array(current_pos) + np.array(action)

        # normally, reward = 0
        reward = 0

        # if new pos results in off the grid, return reward -1
        if -1 in self.new_pos or self.size in self.new_pos:
            reward = -1
        # if in state A, receive + 10
        if current_pos == [0, 1]:
            reward = 10
        # if in state B, receive + 5
        if current_pos == [0, 3]:
            reward = 5

        # if taking an action crosses the border; agent's new_pos is the same as the current pos
        if -1 in self.new_pos or self.size in self.new_pos: 
            self.new_pos = current_pos
            
        # if in state A, transition to state A'
        if current_pos == [0, 1]:
            self.new_pos = [4, 1]
            
        # if in state B, transition to state B'
        if current_pos == [0, 3]:
            self.new_pos = [2, 3]

        return self.new_pos, reward

## First-visit MC Control 

In [4]:
# create a grid object
grid = Gridworld(5)

In [5]:
# Initiate a random policy
random_policy = np.random.randint(1000, size=(state_count, action_count))
random_policy = random_policy/random_policy.sum(axis=1)[:,None]
policy = random_policy

In [6]:
# random policy
policy

array([[0.12514899, 0.28923321, 0.32379817, 0.26181963],
       [0.42983752, 0.02363368, 0.38453964, 0.16198917],
       [0.12077827, 0.23604993, 0.3366373 , 0.30653451],
       [0.19861555, 0.16613419, 0.22523962, 0.41001065],
       [0.27924945, 0.16997792, 0.35154525, 0.19922737],
       [0.2970297 , 0.37699924, 0.16336634, 0.16260472],
       [0.12155213, 0.13604488, 0.43899018, 0.30341281],
       [0.0605042 , 0.21554622, 0.40798319, 0.31596639],
       [0.2893294 , 0.11767187, 0.40531421, 0.18768452],
       [0.17202729, 0.19785575, 0.1837232 , 0.44639376],
       [0.20603774, 0.37962264, 0.36075472, 0.05358491],
       [0.20408163, 0.07849294, 0.32967033, 0.3877551 ],
       [0.41134113, 0.21062106, 0.25382538, 0.12421242],
       [0.42792109, 0.31006576, 0.05159332, 0.21041983],
       [0.22417488, 0.26660952, 0.15345049, 0.35576511],
       [0.27791986, 0.37468031, 0.19352089, 0.15387894],
       [0.16749887, 0.07217431, 0.33091239, 0.42941443],
       [0.30540897, 0.51912929,

### Create an Episode following Policy

In [7]:
def generate_episode(steps):

    # set initial state
    state_vector = grid.initial_state()

    # initialize state (with iniitial state), action list and reward list
    state_list = [state_vector]
    action_list = []
    reward_list = []

    # generate an episode
    for i in range(steps):

        # pick an action based on categorical distribution in policy
        action_index = int(np.random.choice(action_count, 1, p=policy[grid.states.index(state_vector)])) 
        action_vector = actions[action_index] # convert the integer index (ie. 0) to action (ie. [-1, 0])

        # get new state and reward after taking action from current state
        new_state_vector, reward = grid.transition_reward(state_vector, action_vector)
        state_vector = list(new_state_vector)

        # save state, action chosen and reward to list
        state_list.append(state_vector)
        action_list.append(action_vector)
        reward_list.append(reward)
        
    return state_list, action_list, reward_list

### First Visit MC

In [8]:
# initialize q values for all state action pairs
Q_values = np.zeros((state_count, action_count))

In [9]:
# intialize parameters
gamma = 0.99
epsilon = 0.2

In [10]:
# define average function
def Average(lst): 
    return sum(lst) / len(lst) 

In [11]:
# iterate 500 times: each time, generating an episode of 200 steps
max_steps = 200

# define variables for keeping track of time steps
Terminal = max_steps
t_list=[]
for i in range(1,max_steps+1):
    t = Terminal - i
    t_list.append(t)

In [12]:
# iteration 500 times
for iteration in range(500):
  
    # generate an episode of specified step count
    state_list, action_list, reward_list = generate_episode(max_steps)
    
#     print("state_list: ", state_list)
#     print("action_list: ", action_list)
#     print("reward_list: ", reward_list)
    
    # intialize G
    G = 0
    
    # initiate returns and visited list to none
    returns_list = []
    visited_list = []

    # loop for each step of episode: T-1, T-2, T-3 ... 0 = 199, 198, 197 ... 0
    for t in t_list:

        # calculate G: starting with the last reward at index t (naturally accounts for pseudocode's "t-1")
        G = gamma*G + reward_list[t]
        # print(G)
        
        # combine state action pair, for example, state = [0,0], action = [0,1], state_action_pair = [0,0,0,1]
        state_action_pair = []
        state_action_pair.extend(state_list[t])
        state_action_pair.extend(action_list[t])
        # print(state_action_pair)

        # check if state action pair have been visited before (if not: continue, else: move to the next time step)
        if state_action_pair not in visited_list:

            # add state action pair to visited list
            visited_list.append(state_action_pair)

            # append G to returns
            returns_list.append(G)

            # find state and action index, for example, converting action [-1, 0] to 0, and same for state #
            state_index = grid.states.index(state_list[t])
            action_index = actions.index(action_list[t])
    #         print("state_index: ", state_index)

            # write Q_values to the state-action pair
            Q_values[state_index][action_index] = Average(returns_list)
    #         print("average return: ", Average(returns_list))

            # choose best action at given state
            choose_action = np.argmax(Q_values[state_index])

            # overwrite policy
            for a in range(action_count): # for action in actions [0, 1, 2, 3]
                if choose_action == a: # if the choose_action is the same as the current action
                    policy[state_index][a] = 1 - epsilon + epsilon/action_count 
                else: # if choose_action is not the same as the current action
                    policy[state_index][a] = epsilon/action_count

In [13]:
# total unique state action pairs at the end of one episode
print(len(visited_list))

29


In [14]:
Q_values

array([[ 1.33635751e+00,  9.64007527e-01, -4.80968072e+00,
        -5.25534815e+00],
       [-7.44869716e+00, -2.77053487e+00, -1.65205150e+01,
        -1.76672317e+01],
       [-1.00000000e+00, -8.96906046e+00,  9.73461027e+00,
        -1.75772396e+00],
       [-8.94994963e+00,  1.65008317e-02,  1.25341756e+01,
         1.56481833e+01],
       [-1.41166492e+01, -4.14837661e+00, -1.48893507e+01,
        -1.39088985e+01],
       [-5.90783952e+00, -6.98766962e+00,  0.00000000e+00,
        -2.10543289e+01],
       [-1.64013779e+01,  8.03266448e+00,  1.20682295e+01,
        -6.47166459e+00],
       [ 1.03954757e+01,  6.92208782e+00,  1.14494033e+01,
         8.95680703e+00],
       [ 5.85430706e-01, -1.50631460e+01, -7.42525000e-01,
         0.00000000e+00],
       [-1.44881736e+01, -1.48605790e+01, -3.02144205e+01,
        -1.50016705e+01],
       [ 0.00000000e+00, -1.66617694e+01,  0.00000000e+00,
        -1.61283166e+01],
       [ 1.17808436e+01, -1.90625012e+01,  0.00000000e+00,
      

In [15]:
#up, right, down, left = (clockwise from up) 
policy

array([[0.85, 0.05, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.05, 0.05, 0.85],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.85, 0.05, 0.05, 0.05]])

## Visualize 

In [16]:
# PRINT POLICY TABLE ################################################################################
# import pandas library
import pandas as pd
# define column and index
columns=range(grid.size)
index = range(grid.size)
# define dataframe to represent policy table
policy_table = pd.DataFrame(index = index, columns=columns)

# iterate through policy to make a table that represents action number
# as action name (eg. left, right, up, down)
for state in range(len(policy)):
    
    # find the best action at each state
    best_action = np.argmax(policy[state])

    # get action name
    if best_action == 0:
        action_name = 'up'
    elif best_action == 1:
        action_name = 'right'
    elif best_action == 2:
        action_name = 'down'
    else:
        action_name = 'left'

    # calculate the row and column coordinate of the current state number
    row = int(state/grid.size)
    column = round((state/grid.size - int(state/grid.size))*grid.size)
            
    # assign action name
    policy_table.loc[row][column] = action_name

print("Policy Table: ")
print(policy_table)
print()

Policy Table: 
       0      1      2     3      4
0     up  right   down  left  right
1   down   down   down    up     up
2     up     up     up  down   down
3  right   down  right  down  right
4     up  right     up  down     up

